In [1]:
from IPython.display import display, Markdown, Code
import tempfile
from pathlib import Path, PurePath
import os
import subprocess
import shlex
import hashlib
import zlib
from typing import Mapping, Tuple, Literal, Optional

In [2]:
base = tempfile.mkdtemp(prefix='git-mock-')

display(Code(f'cd {base}'))

cd /tmp/git-mock-_87_4ks6

In [3]:
init_empty_dirs = [
    'branches',
    'hooks',
    'info',
    'objects/info',
    'objects/pack',
    'refs/heads',
    'refs/tags'
]

git_dir = PurePath(base).joinpath('.git')
os.mkdir(git_dir)

for d in init_empty_dirs:
    os.makedirs(git_dir.joinpath(d), exist_ok=True)

In [4]:
with open(git_dir.joinpath('config'), 'wt', encoding='ascii') as f:
    f.write('''\
[core]
    repositoryformatversion = 0
    filemode = true
    base = false
    logallrefupdates = true
''')

In [5]:
with open(git_dir.joinpath('HEAD'), 'wt', encoding='ascii') as f:
    f.write('ref: refs/heads/master')

In [6]:
def run_cmd(cmd):
    proc = subprocess.run(shlex.split(cmd), capture_output=True, encoding='ascii')

    display(Code(f'>>> {shlex.join(proc.args)}\n{proc.stdout}'))

In [7]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-_87_4ks6 status
On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)

In [8]:
def write_object(raw_content: bytes, sha1: str, git_dir: PurePath) -> None:
    compressed = zlib.compress(raw_content)
    object_dir = git_dir.joinpath('objects', sha1[:2])
    os.makedirs(object_dir, exist_ok=True)
    with open(object_dir.joinpath(sha1[2:]), 'wb') as f:
        f.write(compressed)

def write_blob_object(file_content: str) -> str:
    raw_content = f'blob {len(file_content)}\0{file_content}'.encode('ascii')
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
        
    return sha1

In [9]:
blob_sha = write_blob_object('very first file 1')
print(blob_sha)

2a94aaff68840af828318ce66927ef8782d9d5dd


In [10]:
run_cmd(f'git -C {base} cat-file -p {blob_sha}')

>>> git -C /tmp/git-mock-_87_4ks6 cat-file -p 2a94aaff68840af828318ce66927ef8782d9d5dd
very first file 1

In [11]:
TreeType = Mapping[str, Tuple[Literal['tree', 'blob'], str]]
def write_tree_object(tree: TreeType) -> str:
    sorted_keys = sorted(tree.keys())
    entries = [{
        'mode': 100644 if tree[k][0] == 'blob' else 40000,
        'name': k,
        'sha1': tree[k][1],
    }
        for k in sorted_keys]
    
    entries_content = b''.join([
        f'{e["mode"]} {e["name"]}\0'.encode('ascii') + bytes.fromhex(e["sha1"]) for e in entries
    ])
    raw_content = f'tree {len(entries_content)}\0'.encode('ascii') + entries_content
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
    
    return sha1

In [12]:
tree_sha = write_tree_object({ 'first.md': ('blob', blob_sha)})
print(tree_sha)

866663a19922bf6851b415da2220ff75a3ba0e06


In [13]:
run_cmd(f'git -C {base} cat-file -p {tree_sha}')

>>> git -C /tmp/git-mock-_87_4ks6 cat-file -p 866663a19922bf6851b415da2220ff75a3ba0e06
100644 blob 2a94aaff68840af828318ce66927ef8782d9d5dd	first.md

In [14]:
def write_commit_object(tree_sha: str, parent_commmit_sha: Optional[str], msg: str) -> str:
    commit = f'tree {tree_sha}\n' + \
        (f'parent {parent_commmit_sha}\n' if parent_commmit_sha else '') + \
        'author Soros Liu <soros.liu1029@gmail.com> 1634271729 +0800\n' + \
        'committer Soros Liu <soros.liu1029@gmail.com> 1634271729 +0800\n' + \
        '\n' + \
        msg + \
        '\n'
    
    commit_content = commit.encode('ascii')
    raw_content = f'commit {len(commit_content)}\0'.encode('ascii') + commit_content
    
    sha1 = hashlib.sha1(raw_content).hexdigest()
    
    write_object(raw_content, sha1, git_dir)
    
    return sha1

In [15]:
commit_sha = write_commit_object(tree_sha, None, 'first commit')
print(commit_sha)

a43e8fd97ffb060145ffe9206d4b46702ee064da


In [16]:
run_cmd(f'git -C {base} cat-file -p {commit_sha}')

>>> git -C /tmp/git-mock-_87_4ks6 cat-file -p a43e8fd97ffb060145ffe9206d4b46702ee064da
tree 866663a19922bf6851b415da2220ff75a3ba0e06
author Soros Liu <soros.liu1029@gmail.com> 1634271729 +0800
committer Soros Liu <soros.liu1029@gmail.com> 1634271729 +0800

first commit

In [17]:
with open(git_dir.joinpath('refs', 'heads', 'master'), 'wt', encoding='ascii') as f:
    f.write(commit_sha)

In [18]:
run_cmd(f'git -C {base} log')

>>> git -C /tmp/git-mock-_87_4ks6 log
commit a43e8fd97ffb060145ffe9206d4b46702ee064da
Author: Soros Liu <soros.liu1029@gmail.com>
Date:   Fri Oct 15 12:22:09 2021 +0800

    first commit

In [19]:
run_cmd(f'git -C {base} status')

>>> git -C /tmp/git-mock-_87_4ks6 status
On branch master
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	deleted:    first.md